In [ ]:
%load_ext autoreload
%autoreload 2

import h5py
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('presentation')
from scipy.ndimage import gaussian_filter
from scipy.signal import savgol_filter
from shabanipy.jj.utils import (correct_voltage_offset,
                                compute_resistance)
from shabanipy.jj.shapiro.utils import correct_voltage_offset_per_power
from shabanipy.jj.shapiro.binning import bin_power_shapiro_steps,extract_step_weight
from shabanipy.jj.shapiro.plotting_shapiro import (plot_differential_resistance_map,
                                           plot_shapiro_histogram,
                                           plot_step_weights)

#: Name of the sample that must appear in the measurement name usually of the form "{Wafer}-{Piece}_{Design}-{Iteration}_{Junction}_{Cooldown}"
SAMPLE_NAME = "{Wafer}-{Piece}_{Design}-{Iteration}"
SAMPLE_ID = "{Wafer}-{Piece}_{Design}-{Iteration}_{Junction}_{Cooldown}"

#: hdf5 file number
FILE_NUM = ''

#: Path to store generated files
PATH = (f"/Users/bh/Desktop/Code/Topological JJ/Samples/{SAMPLE_NAME}/{SAMPLE_ID}")

#: Name of generated processed data file
PROCESSED_DATA_NAME = (f"{PATH}/Data/{SAMPLE_ID}_processed-data-{FILE_NUM}.hdf5")

h = h5py.File(PROCESSED_DATA_NAME, 'r')

temp = 'Temperature::'
field_y = 'In-plane Field - Y::'
field_z = 'In-plane Field - Z::'
freq = 'Microwave: Frequency::'
freq_val = float(freq[22:])
vg = 'Vg::'

f = h['Data'][f'{freq}&{temp}'][f'{field_y}&{field_z}'][f'{vg}']

In [ ]:
data = np.array(f["Voltage Drop"])
scaled_v_drop = np.array(f["ScaledVoltage"])
bias = np.array(f["Bias"])
dVdI = np.diff(np.array(f["ScaledVoltage"]))/np.diff(np.array(f["Bias"]))
dR = np.array(f["dR"])
power = np.array(f["Microwave - Power"])

In [ ]:
coarse_corrected_data = correct_voltage_offset(bias,scaled_v_drop,
                                               n_peak_width = 2, # To identify the zero voltage region the algorithm computes the derivative and look for peaks signaling the transition out of the superconducting domain. This parameter specifies how many width of the peaks to ignore when determining the actual zero voltage region.
#                                                bound = , # Bounds around midpoint to look for peaks (in uA)
#                                                index = , # Index to select only a single trace to use to determine the offset.
                                              )
fine_corrected_data = correct_voltage_offset_per_power(power,bias, coarse_corrected_data,
                                                       frequency = freq_val,
                                                       n_peak_width = 2, # Same as above
                                                       n_std_as_bin = 1, # Number of standard deviation (as determined from the superconducting plateau of the lowest power measurement).
#                                                      bound = , # Bounds around midpoint to look for peaks (in uA)
                                                        )
dr_bias, dr = compute_resistance(bias,fine_corrected_data)

In [ ]:
# power_offset = 

plot_differential_resistance_map(power, bias, np.abs(dR),
#                                  savgol_windowl = 3, savgol_polyorder = 1,
#                                  power_offset = power_offset,
#                                  cvmax = , cvmin = ,
#                                  power_limits = ,
#                                  bias_limits = ,
#                                  fig_size = ,
#                                  transpose = ,
                                )

plt.savefig(f'Figs/Shapiro/dR-map_{SAMPLE_ID}_freq:{freq[22:]}_field-y:{field_y[16:]}_field-z:{field_z[16:]}_Vg:{vg[4:]}_{FILE_NUM}.pdf', bbox_inches = 'tight')

In [ ]:
voltage_bins, histo_counts = bin_power_shapiro_steps(power, bias, fine_corrected_data, 
                                                     frequency = freq_val,
                                                     step_fraction = 0.01 # Fraction of a shapiro step to use when binning the data.
                                                    )

I_c =

plot_shapiro_histogram(power, voltage_bins, histo_counts,
                       I_c = I_c,
#                        savgol_windowl = 3, savgol_polyorder = 1,
#                        power_offset = power_offset,
#                        cvmax = , cvmin = ,
#                        power_limits = ,
#                        bias_limits = ,
#                        mark_steps = [0,-1,-2],
#                        mark_steps_limit = ,
#                        fig_size = ,
#                        transpose = True,
                      )

plt.savefig(f'Figs/Shapiro/histo-map_{SAMPLE_ID}_freq:{freq[22:]}_field-y:{field_y[16:]}_field-z:{field_z[16:]}_Vg:{vg[4:]}_{FILE_NUM}.pdf', bbox_inches = 'tight')

In [ ]:
voltage_bins, histo_counts = bin_power_shapiro_steps(power, bias, fine_corrected_data, 
                                                     frequency = freq_val,
                                                     step_fraction = 0.01 # Fraction of a shapiro step to use when binning the data.
                                                    )

plot_step_weights(power, voltage_bins, histo_counts,
                  steps = [-3,-2,-1,0,1,2,3],
                  ic = I_c*1e-6,
#                   rn = ,
#                   savgol_windowl = 3, savgol_polyorder = 1,
#                   power_offset = power_offset,
#                   counts_limits = 
#                   power_limits = ,
#                   fig_size = ,
                    )

plt.savefig(f'Figs/Shapiro/steps-weights_{SAMPLE_ID}_freq:{freq[22:]}_field-y:{field_y[16:]}_field-z:{field_z[16:]}_Vg:{vg[4:]}_{FILE_NUM}.pdf', bbox_inches = 'tight')